In [1]:
!echo $USER | sudo -S systemctl restart nvargus-daemon 

[sudo] password for jetson: 

In [2]:
%cd csc-3002-efficient-autonomous-obstacles-avoidance-on-jetracer-1-2025/notebooks/train_collision_avoidance

/home/jetson/csc-3002-efficient-autonomous-obstacles-avoidance-on-jetracer-1-2025/notebooks/train_collision_avoidance


# Data Collection
- This file is used to collect data for training the model.
- The data labels are divided into three categories —— `forward`, `turn left` and `turn right`.
- We will store the data in three different folders

In [3]:
import ipywidgets as widgets
from IPython.display import display
import cv2
import os

forward_dir = 'js_data/0_forward'
turnLeft_dir = 'js_data/1_left'
turnRight_dir = 'js_data/2_right'

# Create the folder
try:
    os.makedirs(forward_dir)
    os.makedirs(turnLeft_dir)
    os.makedirs(turnRight_dir)
except FileExistsError:
    print('Directories not created because they already exist')

# Set and display the button
button_layout = widgets.Layout(width='128px', height='64px')
forward_button = widgets.Button(description='add forward', button_style='success', layout=button_layout)
turnLeft_button = widgets.Button(description='add turn left', button_style='danger', layout=button_layout)
turnRight_button = widgets.Button(description='add turn right', button_style='danger', layout=button_layout)

# Count the photos
forward_count = widgets.IntText(layout=button_layout, value=len(os.listdir(forward_dir)))
turnLeft_count = widgets.IntText(layout=button_layout, value=len(os.listdir(turnLeft_dir)))
turnRight_count = widgets.IntText(layout=button_layout, value=len(os.listdir(turnRight_dir)))

Directories not created because they already exist


### Initialize the camera

In [4]:
import traitlets 
import ipywidgets.widgets as widgets
from IPython.display import display
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_fps=65)
camera.running = True

image_widget = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

### Start collecting data
- Move the car to the scene you want to shoot, click the corresponding button, and the image will be automatically saved to the corresponding folder.

In [5]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    cv2.imwrite(image_path, camera.value.copy())

def save_forward():
    global forward_dir, forward_count
    save_snapshot(forward_dir)
    forward_count.value = len(os.listdir(forward_dir))

def save_turnLeft():
    global turnLeft_dir, turnLeft_count
    save_snapshot(turnLeft_dir)
    turnLeft_count.value = len(os.listdir(turnLeft_dir))

def save_turnRight():
    global turnRight_dir, turnRight_count
    save_snapshot(turnRight_dir)
    turnRight_count.value = len(os.listdir(turnRight_dir))
    
forward_button.on_click(lambda x: save_forward())
turnLeft_button.on_click(lambda x: save_turnLeft())
turnRight_button.on_click(lambda x: save_turnRight())

## Display all together

In [6]:
display(widgets.HBox([forward_count, forward_button]))
display(widgets.HBox([turnLeft_count, turnLeft_button]))
display(widgets.HBox([turnRight_count, turnRight_button]))
display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…